<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/codecompletion_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Modify torch and transformers which requires manual notebook restart
# Could not redirect to /dev/null in the standard Colab notebook (maybe no output for a particular time?)
# Currently deepspeed from GTP-NeoX is not compatible with logging in torch >= 2.4
%pip install torch==2.3 torchaudio==2.3.0 torchvision==0.18.0 transformers==4.38.0 sentence-transformers==2.2.2

In [34]:
!apt install -y sysbench
# The Colab vCPU should give around 250 evenst per second
!sysbench cpu --cpu-max-prime=20000 run

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sysbench is already the newest version (1.0.20+ds-2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
sysbench 1.0.20 (using system LuaJIT 2.1.0-beta3)

Running the test with following options:
Number of threads: 1
Initializing random number generator from current time


Prime numbers limit: 20000

Initializing worker threads...

Threads started!

CPU speed:
    events per second:   153.57

General statistics:
    total time:                          10.0001s
    total number of events:              1536

Latency (ms):
         min:                                    2.79
         avg:                                    6.50
         max:                                   33.86
         95th percentile:                       16.71
         sum:                                 9991.31

Threads fairness:
    events (avg/stddev):           1536.0000/0.00
    execution time (avg

In [1]:
# We could modify these paths to "stub" behavior for test/dev
workspaceDir = "/content"
GPTNeoXDirName = "gpt-neox"
GPTNeoXDir = f"{workspaceDir}/{GPTNeoXDirName}"
GPTNeoXColabDirName = "GPT-NeoX-Colab"
GPTNeoXColabDir = f"{workspaceDir}/{GPTNeoXColabDirName}"

# Clone CodeXGLUE Repo

In [2]:
!git clone https://github.com/microsoft/CodeXGLUE.git

Cloning into 'CodeXGLUE'...
remote: Enumerating objects: 3373, done.
remote: Counting objects: 100% (3372/3372), done.
remote: Compressing objects: 100% (1534/1534), done.
remote: Total 3373 (delta 1748), reused 3326 (delta 1733), pack-reused 1 (from 1)
Receiving objects: 100% (3373/3373), 213.15 MiB | 7.84 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (400/400), done.


In [3]:
#@title Clone GPT-NeoX-Colab
%%time
%cd {workspaceDir}
# Don't use --depth 1 because that does not play nice with git-annex
!git clone https://github.com/markNZed/GPT-NeoX-Colab.git
%cd {GPTNeoXColabDir}
%pip install -q -r requirements_colab.txt
%pip install -q .
from dotenv import load_dotenv
import os
load_dotenv(f"{GPTNeoXColabDir}/.env")
import GPTNeoXColab
GPTNeoXColab.utils.colab.fetch_data("data/codecompletion/processed_data")

/content
Cloning into 'GPT-NeoX-Colab'...
remote: Enumerating objects: 1331, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 1331 (delta 169), reused 137 (delta 65), pack-reused 1052 (from 1)
Receiving objects: 100% (1331/1331), 13.90 MiB | 12.49 MiB/s, done.
Resolving deltas: 100% (746/746), done.
/content/GPT-NeoX-Colab
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 76.9 MB/

In [18]:
%cd {workspaceDir}
#!git clone https://github.com/EleutherAI/gpt-neox.git
!git clone -b pipe_parallel_size_1 --depth 1 https://github.com/markNZed/gpt-neox.git

/content
Cloning into 'gpt-neox'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (296/296), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 296 (delta 74), reused 138 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (296/296), 2.50 MiB | 6.46 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [2]:
!mkdir -p /content/gpt-neox/processed_data
!cp {GPTNeoXColabDir}/data/codecompletion/processed_data/* /content/gpt-neox/processed_data
processed_data_path = "/content/gpt-neox/processed_data"

# Cloning GPT-NeoX Repo

# Downloading and Preprocessing Dataset

In [21]:
import os

# Check if the file exists
if not os.path.exists(processed_data_path):
    # Change directory
    %cd /content/CodeXGLUE/Code-Code/CodeCompletion-token/dataset/py150
    # Run the shell script to download and extract
    !bash /content/CodeXGLUE/Code-Code/CodeCompletion-token/dataset/py150/download_and_extract.sh
    # Run the preprocessing Python script
    !python preprocess.py --base_dir=py150_files --output_dir=token_completion
else:
    print("File already exists, skipping download and preprocessing.")


File already exists, skipping download and preprocessing.


# Install Dependencies

In [22]:
%%time
%cd /content/gpt-neox
%pip install -r ./requirements/requirements.txt

/content/gpt-neox
  Using cached torch-2.3.0-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchaudio-2.3.0-cp310-cp310-manylinux1_x86_64.whl.metadata (6.4 kB)
  Using cached torchvision-0.18.0-cp310-cp310-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached transformers-4.41.0-py3-none-any.whl.metadata (43 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached torch-2.3.0-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
Using cached torchaudio-2.3.0-cp310-cp310-manylinux1_x86_64.whl (3.3 MB)
Using cached torchvision-0.18.0-cp310-cp310-manylinux1_x86_64.whl (7.0 MB)
Using cached transformers-4.41.0-py3-none-any.whl (9.1 MB)
Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
Using cached tokenizers-0.19.1-cp310-cp310-ma

  Cloning https://github.com/EleutherAI/DeeperSpeed.git (to revision 02e2ebf7dee6aaab3d89094ed470a4609763c742) to /tmp/pip-install-njroaev_/deepspeed_978af396afe3443cac091073bd466dfc
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/DeeperSpeed.git /tmp/pip-install-njroaev_/deepspeed_978af396afe3443cac091073bd466dfc
  Running command git rev-parse -q --verify 'sha^02e2ebf7dee6aaab3d89094ed470a4609763c742'
  Running command git fetch -q https://github.com/EleutherAI/DeeperSpeed.git 02e2ebf7dee6aaab3d89094ed470a4609763c742
  Running command git checkout -q 02e2ebf7dee6aaab3d89094ed470a4609763c742
  Resolved https://github.com/EleutherAI/DeeperSpeed.git to commit 02e2ebf7dee6aaab3d89094ed470a4609763c742
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/EleutherAI/lm_dataformat.git (to revision 4eec05349977071bf67fc072290b95e31c8dd836) to /tmp/pip-install-njroaev_/lm-dataform

CPU times: user 3.98 s, sys: 564 ms, total: 4.54 s
Wall time: 8min 33s


# Preparing Custom Dataset


In [3]:
%cd /content/gpt-neox
!mkdir -p data

/content/gpt-neox


In [4]:
import json
import os

# Generate a list of dictionaries
if not os.path.exists(processed_data_path):
  lines = []
  with open("/content/CodeXGLUE/Code-Code/CodeCompletion-token/dataset/py150/token_completion/train.txt", encoding="utf8") as f:
      for line in f.read().splitlines():
          if line:
              lines.append({"text": line})

  # Convert to a list of JSON strings
  json_lines = [json.dumps(l) for l in lines]

  # Join lines and save to .jsonl file
  json_data = '\n'.join(json_lines)
  with open('/content/gpt-neox/data/py95K_train.jsonl', 'w') as f:
      f.write(json_data)

# Using Byte-Pair Encoding Tokenizer

In [5]:
%cd data
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt

/content/gpt-neox/data
--2024-11-14 07:51:49--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.209.152, 52.216.244.174, 52.217.207.32, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.209.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving to: ‘gpt2-vocab.json.1’

gpt2-vocab.json.1   100%[===================>]   1018K  2.45MB/s    in 0.4s    

2024-11-14 07:51:50 (2.45 MB/s) - ‘gpt2-vocab.json.1’ saved [1042301/1042301]

--2024-11-14 07:51:50--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.107.230, 52.216.164.53, 52.216.217.56, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.107.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: ‘gpt2-merges.txt.1’

gpt2-merges.txt.1   100%[===

In [13]:
%%time
import os

# Check if the file exists
if not os.path.exists(processed_data_path):
  %cd /content/gpt-neox
  !mkdir -p processed_data
  !python tools/datasets/preprocess_data.py \
    --input ./data/py95K_train.jsonl \
    --vocab ./data/gpt2-vocab.json \
    --merge-file ./data/gpt2-merges.txt \
    --output-prefix ./processed_data/py150 \
    --tokenizer-type GPT2BPETokenizer \
    --dataset-impl mmap \
    --append-eod
else:
    print("File already exists, skipping download and preprocessing.")


File already exists, skipping download and preprocessing.
CPU times: user 128 µs, sys: 16 µs, total: 144 µs
Wall time: 150 µs


# Tokens count in Dataset

In [14]:
from transformers import GPT2Tokenizer

# Initialize the GPT-2 tokenizer (BPE-based)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Path to your text file
file_path = "/content/CodeXGLUE/Code-Code/CodeCompletion-token/dataset/py150/token_completion/train.txt"

# Initialize a token counter
total_token_count = 0

# Open the file and read line by line to count tokens
#with open(file_path, "r", encoding="utf-8") as file:
#    for line in file:
#        tokens = tokenizer.encode(line)
#        total_token_count += len(tokens)
        #print(total_token_count)

print(f"Total token count: {total_token_count}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Total token count: 0


# Training

In [15]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [20]:
%%time
%cd /content/gpt-neox
!python ./deepy.py train.py --conf_dir /content/GPT-NeoX-Colab/configs codecompletion codecompletion_train

/content/gpt-neox
[2024-11-14 07:54:33,269] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
NeoXArgs.from_ymls() ['/content/GPT-NeoX-Colab/configs/codecompletion.yml', '/content/GPT-NeoX-Colab/configs/codecompletion_train.yml']
INFO:root:NeoXArgs.calculate_derived() Total number of GPUs determined to be: 0
Traceback (most recent call last):
  File "/content/gpt-neox/./deepy.py", line 41, in <module>
    main()
  File "/content/gpt-neox/./deepy.py", line 28, in main
    neox_args = NeoXArgs.consume_deepy_args(input_args)
  File "/content/gpt-neox/megatron/neox_arguments/arguments.py", line 425, in consume_deepy_args
    neox_args = cls.from_ymls(
  File "/content/gpt-neox/megatron/neox_arguments/arguments.py", line 283, in from_ymls
    return cls(**config)
  File "<string>", line 312, in __init__
  File "/content/gpt-neox/megatron/neox_arguments/arguments.py", line 148, in __post_init__
    self.calculate_derived()
  File "/content/gpt-neox

In [17]:
!pip show datasets
!pip install datasets==1.18.0
!pip install hf-transfer
!pip install lm-eval --upgrade

Name: datasets
Version: 3.1.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: evaluate, lm_eval
  Using cached datasets-1.18.0-py3-none-any.whl.metadata (22 kB)
Using cached datasets-1.18.0-py3-none-any.whl (311 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 3.1.0
    Uninstalling datasets-3.1.0:
      Successfully uninstalled datasets-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
evaluate 0.4.3 requires datasets>=2.0.0, but you have datasets 1.18.0 which is incompatible.

# Inference

In [18]:
%cd /content/gpt-neox

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator
import os
import numpy as np
# Path to the latest log file
log_dir = "tensorboard"
log_files = [os.path.join(log_dir, d) for d in os.listdir(log_dir)]
latest_log_dir = max(log_files, key=os.path.getmtime)

# Initialize EventAccumulator to load scalar data
ea = event_accumulator.EventAccumulator(latest_log_dir)
ea.Reload()  # Load all logs

# List all scalar keys available in the logs
scalar_keys = ea.Tags()['scalars']
print("Available scalar keys:", scalar_keys)

# Extract training and validation losses
train_loss = ea.Scalars('train/lm_loss')  # Adjust for actual name if necessary
val_loss = ea.Scalars('validation/lm_loss')  # Adjust for actual name if necessary

# Convert to lists for plotting
train_loss_values = [x.value for x in train_loss]
val_loss_values = [x.value for x in val_loss]

# Find the lengths of both arrays
len_train = len(train_loss_values)
len_val = len(val_loss_values)

iterations = None
# Interpolate the shorter array
if len_train != len_val:
    if len_train > len_val:
        # Interpolate validation loss to match the training loss length
        iterations = np.linspace(1, len_train, len_train)
        val_iterations = np.linspace(1, len_train, len_val)
        val_loss_values = np.interp(iterations, val_iterations, val_loss_values)
    else:
        # Interpolate training loss to match the validation loss length
        iterations = np.linspace(1, len_val, len_val)
        train_iterations = np.linspace(1, len_val, len_train)
        train_loss_values = np.interp(iterations, train_iterations, train_loss_values)
else:
    iterations = range(1, len_train + 1)

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(iterations, train_loss_values, label='Training Loss')
plt.plot(iterations, val_loss_values, label='Validation Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


/content/gpt-neox


FileNotFoundError: [Errno 2] No such file or directory: 'tensorboard'

# HuggingFace Inference

# Convert Our Model to HuggingFace Format

In [ ]:
import os

# Define the path to the checkpoints directory
checkpoints_dir = "/content/gpt-neox/checkpoints"

# Read the 'latest' file to get the latest checkpoint name
with open(os.path.join(checkpoints_dir, "latest"), "r") as f:
    latest_checkpoint_name = f.read().strip()

# Construct the full path to the latest checkpoint directory
latest_checkpoint_path = os.path.join(checkpoints_dir, latest_checkpoint_name)
print("Path to the latest checkpoint:", latest_checkpoint_path)

In [ ]:
!python ./tools/ckpts/convert_neox_to_hf.py --input_dir {latest_checkpoint_path} --config_file /content/GPT-NeoX-Colab/configs/codecompletion.yml --output_dir hf_model/save/location --precision auto --architecture neox

# Code Completion

In [ ]:
from transformers import GPTNeoXForCausalLM
import torch

# Move to model directory
%cd /content/gpt-neox

# Assuming CharLevelTokenizer is properly imported and instantiated
from megatron.tokenizer.tokenizer import _GPT2BPETokenizer
tokenizer = _GPT2BPETokenizer(vocab_file="data/gpt2-vocab.json", merge_file="data/gpt2-merges.txt")

# Load your model
model_path = "/content/gpt-neox/hf_model/save/location"
model = GPTNeoXForCausalLM.from_pretrained(model_path)

# Define a simple char-level tokenizer if not provided
def token_level_tokenize(text):
    return tokenizer.tokenize(text)

def token_level_detokenize(tokens):
    return tokenizer.detokenize(tokens)

# Set the model to evaluation mode
model.eval()

# Prompt the user for input
input_text = """<s> import sys , os <EOL> import imp <EOL> from optparse import make_option <EOL> from django . conf import settings <EOL> from django . utils . importlib import import_module <EOL> from django . core . management import call_command <EOL> from django . core . management import BaseCommand <EOL> from django . db import connections <EOL> def import_app ( app_label , verbosity ) : <EOL> try : <EOL> app_path = __import__ ( app_label , { } , { } , [ app_label . split ( '<STR_LIT:.>' ) [ - <NUM_LIT:1> ] ] ) . __path__ <EOL>"""

# Tokenize and prepare input
input_ids = torch.tensor([token_level_tokenize(input_text)], dtype=torch.long)
attention_mask = torch.ones_like(input_ids)  # Create an attention mask for non-padded input

# Generate text with specified pad_token_id and attention_mask
with torch.no_grad():
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=200,          # Adjust this for desired output length
        temperature=0.7,        # Controls creativity
        top_k=50,               # Controls diversity
        top_p=0.9,              # Nucleus sampling
        num_return_sequences=1, # Number of sequences to return
        pad_token_id=model.config.eos_token_id,  # Set pad_token_id explicitly
        do_sample=True           # Enable sampling mode to use temperature and top_p
    )

# Decode the generated text
generated_text = token_level_detokenize(output[0].tolist())

# Function to replace special tokens with original representations
def replace_special_tokens(text):
    replacements = {
        "<EOL>": "\n",  # Replace with actual newline
        "<s>": "",
        "</s>": "",     # Remove end token
        "<STR_LIT>": "STR_LITERAL",  # Example replacement, adjust as necessary
        "<NUM_LIT>": "NUM_LITERAL",   # Example replacement, adjust as necessary
    }

    for token, replacement in replacements.items():
        text = text.replace(token, replacement)

    return text.strip()  # Strip leading/trailing whitespace

# Replace special tokens in the generated text
final_text = replace_special_tokens(generated_text)

# Print the final output
print("Generated text:", final_text)


In [ ]:
!cp /content/gpt-neox/data/gpt2-vocab.json /content/gpt-neox/hf_model/save/location/vocab.json
!cp /content/gpt-neox/data/gpt2-merges.txt /content/gpt-neox/hf_model/save/location/merges.txt
%cd /content/CodeXGLUE/Code-Code/CodeCompletion-token/code
!python -u run_lm.py \
        --data_dir=../dataset/py150/token_completion \
        --lit_file=../dataset/py150/literals.json \
        --langs=$LANG \
        --output_dir=../dataset/py150 \
        --pretrain_dir=/content/gpt-neox/hf_model/save/location \
        --log_file=../completion_python_eval.log \
        --model_type=gpt2neox \
        --block_size=2048 \
        --do_eval \
        --per_gpu_eval_batch_size=4 \
        --logging_steps=100 \
        --seed=42